In [1]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
date = "2021-12"

my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"
# This is my key. I'm nice and I have it posted. If you will be doing more with this
# please get your own key!

In [3]:
end_use = "naics?get=CON_VAL_MO,CTY_CODE,CTY_NAME,SUMMARY_LVL"

url = "https://api.census.gov/data/timeseries/intltrade/imports/" + end_use 
url = url + my_key + "&time==from+2013-01"

r = requests.get(url) 
    
print(r)
    
df = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

df.columns = r.json()[0]

df["total_imports"] = df["CON_VAL_MO"].astype(float)

df = df[df.SUMMARY_LVL == "DET"]

grp = df.groupby(["CTY_NAME"])

top_products = grp.agg({"total_imports":"sum","CTY_CODE":"first"})

country_list = list(top_products.sort_values(by = "total_imports", ascending = False).CTY_CODE)[0:31]


['TOTAL FOR ALL COUNTRIES','NAFTA','EUROPEAN UNION']

<Response [200]>


['TOTAL FOR ALL COUNTRIES', 'NAFTA', 'EUROPEAN UNION']

In [4]:
country_list[0] = ""

In [5]:
country_list

['',
 '5700',
 '2010',
 '1220',
 '5880',
 '4280',
 '5800',
 '4120',
 '5520',
 '5330',
 '4190',
 '4759',
 '4279',
 '5830',
 '4419',
 '5570',
 '5490',
 '3510',
 '5170',
 '5590',
 '4210',
 '5081',
 '4621',
 '5600',
 '4231',
 '4700',
 '3010',
 '4330',
 '3070',
 '5650',
 '4010']

In [7]:
end_use = "hs?get=CTY_NAME,CON_VAL_MO,CAL_DUT_MO,I_COMMODITY,I_COMMODITY_SDESC"

surl = "https://api.census.gov/data/timeseries/intltrade/imports/" + end_use 

surl  = surl + my_key + "&time=" + "from+2013-01" + "&COMM_LVL=HS2" 

for xxx in country_list:
    
    out_file = ".\\data"+ "\\imports\\" + xxx + "data-" + date + ".parquet"
    
    if xxx == "":
        out_file = ".\\data"+ "\\imports\\" + "TOTAL" + "data-" + date + ".parquet"
    
    
    if os.path.exists(out_file):
        
        print("Already have downloaded file")
        
        continue
    
    print(xxx)
    
    url = surl + "&CTY_CODE=" + xxx
    
    if xxx == "":
        url = surl
    
    r = requests.get(url) 
    
    print(r)
    
    foo = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

    foo.columns = r.json()[0]

    pq.write_table(pa.Table.from_pandas(foo), out_file)

Already have downloaded file
5700
<Response [200]>
2010
<Response [200]>
1220
<Response [200]>
5880
<Response [200]>
4280
<Response [200]>
5800
<Response [200]>
4120
<Response [200]>
5520
<Response [200]>
5330
<Response [200]>
4190
<Response [200]>
4759
<Response [200]>
4279
<Response [200]>
5830
<Response [200]>
4419
<Response [200]>
5570
<Response [200]>
5490
<Response [200]>
3510
<Response [200]>
5170
<Response [200]>
5590
<Response [200]>
4210
<Response [200]>
5081
<Response [200]>
4621
<Response [200]>
5600
<Response [200]>
4231
<Response [200]>
4700
<Response [200]>
3010
<Response [200]>
4330
<Response [200]>
3070
<Response [200]>
5650
<Response [200]>
4010
<Response [200]>


In [8]:
country_list[0] = "TOTAL"

df = pd.DataFrame([])
#this is for all countries...

for xxx in country_list:
    
    file = ".\\data\\imports\\" + xxx + "data-" + date + ".parquet"

    foo = pq.read_table(file).to_pandas()
    
    #recentfile = ".\\data\\imports\\" + xxx + "-" + date + "-" + "data.parquet"
    
    #foo = foo.append(pq.read_table(recentfile).to_pandas())
    
    #pq.write_table(pa.Table.from_pandas(foo), file)
    #this combines the two periods
    
    df = df.append(foo)
            
df.time = pd.to_datetime(df.time, format="%Y-%m")

df["imports"] = df["CON_VAL_MO"].astype(float)

df["duty"] = df["CAL_DUT_MO"].astype(float)

total = df.groupby(["CTY_NAME", "time"]).agg({"imports": "sum", "duty": "sum"})
                   
total.reset_index(inplace = True)
                   
total["I_COMMODITY"] = "00"
total["I_COMMODITY_SDESC"] = "ALL PRODUCTS"
                   
df = df.append(total)

df["itariff"] = 100*df["duty"]/df["imports"]

df.drop(["CON_VAL_MO", "COMM_LVL", "CTY_CODE", "CAL_DUT_MO"], axis = 1, inplace = True)

df["I_COMMODITY_SDESC"] = "HS CODE " + df["I_COMMODITY"] + ", " + df["I_COMMODITY_SDESC"]

df["I_COMMODITY_SDESC"].replace({"HS CODE 00, ALL PRODUCTS": "ALL PRODUCTS"}, inplace = True)

df.set_index(["CTY_NAME","I_COMMODITY_SDESC","time"],inplace = True)

out_file = ".\\data"+ "\\top30-HS2-imports.parquet"

pq.write_table(pa.Table.from_pandas(df), out_file)

In [9]:
df.loc["CHINA","ALL PRODUCTS"]["2021"]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


,I_COMMODITY,imports,duty,itariff
time,,,,
2021-01-01,00,3.819920e+10,4.331589e+09,11.339476
2021-02-01,00,3.395794e+10,3.903253e+09,11.494374
2021-03-01,00,4.018032e+10,4.620955e+09,11.500542
2021-04-01,00,3.718709e+10,4.184789e+09,11.253339
2021-05-01,00,3.812285e+10,4.437557e+09,11.640151
2021-06-01,00,3.976041e+10,4.628279e+09,11.640420
2021-07-01,00,3.957695e+10,4.678938e+09,11.822380
2021-08-01,00,4.216437e+10,5.208857e+09,12.353693
2021-09-01,00,4.794939e+10,5.222531e+09,10.891756


In [10]:
file = ".\\data\\imports\\" + "TOTAL" + "data.parquet"

foo = pq.read_table(file).to_pandas()
    
foo

,CTY_NAME,CON_VAL_MO,CAL_DUT_MO,I_COMMODITY,I_COMMODITY_SDESC,time,COMM_LVL
0,TOTAL FOR ALL COUNTRIES,784536648,4504327,07,EDIBLE VEGETABLES & CERTAIN ROOTS & TUBERS,2013-01,HS2
1,TOTAL FOR ALL COUNTRIES,315892605,2650541,17,SUGARS AND SUGAR CONFECTIONARY,2013-01,HS2
2,TOTAL FOR ALL COUNTRIES,334605750,723046,10,CEREALS,2013-01,HS2
3,TOTAL FOR ALL COUNTRIES,144113171,409478,13,"LAC; GUMS, RESINS & OTHER VEGETABLE SAP & EXTRACT",2013-01,HS2
4,TOTAL FOR ALL COUNTRIES,371169570,2283401,18,COCOA AND COCOA PREPARATIONS,2013-01,HS2
...,...,...,...,...,...,...,...
9991,TOTAL FOR ALL COUNTRIES,794906854,2374814,09,"COFFEE, TEA, MATE & SPICES",2021-06,HS2
9992,TOTAL FOR ALL COUNTRIES,146629956,1172022,10,CEREALS,2021-06,HS2
9993,TOTAL FOR ALL COUNTRIES,186372088,2220530,11,MILLING PRODUCTS; MALT; STARCH; INULIN; WHT GL...,2021-06,HS2
9994,TOTAL FOR ALL COUNTRIES,340184476,3851904,12,"OIL SEEDS ETC.; MISC GRAIN, SEED, FRUIT, PLANT...",2021-06,HS2


In [11]:
foo

,CTY_NAME,CON_VAL_MO,CAL_DUT_MO,I_COMMODITY,I_COMMODITY_SDESC,time,COMM_LVL
0,TOTAL FOR ALL COUNTRIES,784536648,4504327,07,EDIBLE VEGETABLES & CERTAIN ROOTS & TUBERS,2013-01,HS2
1,TOTAL FOR ALL COUNTRIES,315892605,2650541,17,SUGARS AND SUGAR CONFECTIONARY,2013-01,HS2
2,TOTAL FOR ALL COUNTRIES,334605750,723046,10,CEREALS,2013-01,HS2
3,TOTAL FOR ALL COUNTRIES,144113171,409478,13,"LAC; GUMS, RESINS & OTHER VEGETABLE SAP & EXTRACT",2013-01,HS2
4,TOTAL FOR ALL COUNTRIES,371169570,2283401,18,COCOA AND COCOA PREPARATIONS,2013-01,HS2
...,...,...,...,...,...,...,...
9991,TOTAL FOR ALL COUNTRIES,794906854,2374814,09,"COFFEE, TEA, MATE & SPICES",2021-06,HS2
9992,TOTAL FOR ALL COUNTRIES,146629956,1172022,10,CEREALS,2021-06,HS2
9993,TOTAL FOR ALL COUNTRIES,186372088,2220530,11,MILLING PRODUCTS; MALT; STARCH; INULIN; WHT GL...,2021-06,HS2
9994,TOTAL FOR ALL COUNTRIES,340184476,3851904,12,"OIL SEEDS ETC.; MISC GRAIN, SEED, FRUIT, PLANT...",2021-06,HS2


In [12]:
foo = pq.read_table(".\\data\\top30-HS2-imports.parquet").to_pandas()

In [13]:
foo.tail()

I_COMMODITY       imports         duty  \
CTY_NAME I_COMMODITY_SDESC time                                                
VIETNAM  ALL PRODUCTS      2021-08-01          00  9.514674e+09  474037492.0   
                           2021-09-01          00  8.654858e+09  394538107.0   
                           2021-10-01          00  8.491478e+09  389929228.0   
                           2021-11-01          00  8.711109e+09  366912083.0   
                           2021-12-01          00  9.323349e+09  397814101.0   

                                        itariff  
CTY_NAME I_COMMODITY_SDESC time                  
VIETNAM  ALL PRODUCTS      2021-08-01  4.982173  
                           2021-09-01  4.558574  
                           2021-10-01  4.592006  
                           2021-11-01  4.212002  
                           2021-12-01  4.266858

In [14]:
foo.loc["TOTAL FOR ALL COUNTRIES"]

I_COMMODITY       imports          duty  \
I_COMMODITY_SDESC        time                                                 
HS CODE 01, LIVE ANIMALS 2013-01-01          01  2.213131e+08  4.170000e+02   
                         2013-02-01          01  2.460921e+08  8.230000e+02   
                         2013-03-01          01  2.612444e+08  7.500000e+01   
                         2013-04-01          01  2.711339e+08  1.389000e+03   
                         2013-05-01          01  2.097853e+08  5.000000e+01   
...                                         ...           ...           ...   
ALL PRODUCTS             2021-08-01          00  2.443148e+11  7.656996e+09   
                         2021-09-01          00  2.451292e+11  7.550079e+09   
                         2021-10-01          00  2.481223e+11  7.665175e+09   
                         2021-11-01          00  2.572687e+11  7.584579e+09   
                         2021-12-01          00  2.589189e+11  7.830120e+09   

                                      itariff  
I_COMMODITY_SDESC        time                  
HS CODE 01, LIVE ANIMALS 2013-01-01  0.000188  
                         2013-02-01  0.000334  
                         2013-03-01  0.000029  
                         2013-04-01  0.000512  
                         2013-05-01  0.000024  
...                                       ...  
ALL PRODUCTS             2021-08-01  3.134070  
                         2021-09-01  3.080041  
                         2021-10-01  3.089273  
                         2021-11-01  2.948116  
                         2021-12-01  3.024159  

[10692 rows x 4 columns]